# Import neccessary packages

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, cKDTree
from shapely.geometry import LineString, Point, MultiLineString, Polygon, MultiPoint
from shapely.ops import linemerge, nearest_points, voronoi_diagram, unary_union, split, substring
from shapely import minimum_rotated_rectangle, unary_union
import networkx as nx
import math
import warnings

warnings.filterwarnings('ignore')

# Import functions

In [2]:
from dist_funcs import *

# Read example data

In [3]:
target_crs = 'EPSG:32736'  # Select a CRS that is in meters

In [4]:
# Read and project the polygons
cluster_polygons = gpd.read_file(r'Input_Data\settlement_polygons.gpkg').to_crs(target_crs)
households = gpd.read_file(r'Input_Data\buildings.gpkg').to_crs(target_crs)

# Example multiple location

In [6]:
import time

# Seelect which settlement polygons tu use based on the 'Site Unique ID (VIDA)_2' column
#fids = [20794, 300, 2460, 2885, 7695, 13999, 3425]
fids = [2460]

multi_trunks = []
multi_trunks_len = []

multi_secondary = []
multi_secondary_len = []

multi_service = []
multi_service_len = []

multi_poles = []
multi_all_poles = []

for fid in fids:
    t1 = time.time()

    # Get the polygon from the gdf of all polygons
    polygon_data = cluster_polygons.loc[cluster_polygons['Site Unique ID (VIDA)_2'] == fid]
    
    # Select the geometry
    polygon = polygon_data.geometry.iloc[0]

    # Create trunk line(s)
    trunk_lines = create_trunk_line(polygon, spacing=100, plot=False)

    # Simplify the trunk line(s)
    trunk_lines, intersects = simplify_trunk_lines(trunk_lines, length_removal=200, split_distance=750, plot=False)

    trunk_lines_gdf = gpd.GeoDataFrame(geometry=trunk_lines)
    
    # Identify which part of the polygon belongs to which part of the trunk line(s)
    voronois = voronoi_areas(trunk_lines_gdf, polygon, plot=False)

    # Parameters for perpendicular lines
    spacing = 50 # Spacing distance between potential candidate poles for buildings to connect to (using target CRS)

    trunk_p = []
    assigned_p = []
    lv_l = []
    service_l = []
    mst_p = []
    all_p = []
    long_services = 0

    # Iterate over each part of the trunk line 
    for id in range(len(trunk_lines)):
        all_poles, trunk_poles, poles, angle_radians_w, angle_radians_l = create_candidate_poles(voronois[id], trunk_lines[id], spacing, buffer=25, plot=False)
    
        trunk_p += trunk_poles
        all_p += all_poles
            
        polygon_households = households.clip(voronois[id])
    
        # Ensure households are not MultiPoint
        polygon_households['geometry'] = polygon_households['geometry'].apply(convert_multipoint_to_point)

        assigned_poles, service_drops = assign_households(all_poles, polygon_households)
    
        assigned_p += all_poles
        service_l += service_drops

        for s in service_drops:
            if s.length > 70:
                long_services += 1
    
        weight = 0.5  # Weighting factor for the MST
    
        lv_lines, mst_poles = lv_lines_mst(all_poles, trunk_poles, assigned_poles, angle_radians_w, angle_radians_l, weight, plot=False)
    
        lv_l += lv_lines
        mst_p += mst_poles

    multi_all_poles.append(MultiPoint(all_p))

    multi_trunks.append(MultiLineString(trunk_lines))
    multi_trunks_len.append(MultiLineString(trunk_lines).length)

    multi_secondary.append(MultiLineString(lv_l))
    multi_secondary_len.append(MultiLineString(lv_l).length)

    multi_service.append(MultiLineString(service_l))
    multi_service_len.append(MultiLineString(service_l).length)

    multi_poles.append(MultiPoint(mst_p))
    
    t2 = time.time()
    print(fid, round((t2-t1)/60, 2), round(MultiLineString(trunk_lines).length, 1), round(MultiLineString(lv_l).length, 1), 
          round(MultiLineString(service_l).length, 1), long_services, len(mst_p))

2460 0.1 3041.1 6918.2 21850.6 0 198


# Export example results

In [8]:
trunks_gdf = gpd.GeoDataFrame()
trunks_gdf['VIDA_id'] = fids
trunks_gdf['Length'] = multi_trunks_len
trunks_gdf.geometry = multi_trunks
trunks_gdf.set_crs(target_crs, inplace=True)
trunks_gdf.to_crs(4326, inplace=True)
trunks_gdf.to_file(r'Output_Data\primary_trunks.geojson')

secondary_gdf = gpd.GeoDataFrame()
secondary_gdf['VIDA_id'] = fids
secondary_gdf['Length'] = multi_secondary_len
secondary_gdf.geometry = multi_secondary
secondary_gdf.set_crs(target_crs, inplace=True)
secondary_gdf.to_crs(4326, inplace=True)
secondary_gdf.to_file(r'Output_Data\secondary_lines.geojson')

service_gdf = gpd.GeoDataFrame()
service_gdf['VIDA_id'] = fids
service_gdf['Length'] = multi_service_len
service_gdf.geometry = multi_service
service_gdf.set_crs(target_crs, inplace=True)
service_gdf.to_crs(4326, inplace=True)
service_gdf.to_file(r'Output_Data\service_drops.geojson')

poles_gdf = gpd.GeoDataFrame()
poles_gdf['VIDA_id'] = fids
poles_gdf.geometry = multi_poles
poles_gdf.set_crs(target_crs, inplace=True)
poles_gdf.to_crs(4326, inplace=True)
poles_gdf.to_file(r'Output_Data\poles.geojson')